6/13 (Thu) | Experiment

# Dominance Analyses of Prediction Models

## 1. Introduction

This notebook evaluate the reliability of the UF measures calculated based on the proposed annotation system. More specifically, I construct linear models to predict PF for each task.

The input variables are follows:
- Speed fluency
    - Articulation rate (AR)
- Breakdown fluency
    - Mid-clause pause ratio (MCPR)
    - End-clause pause ratio (ECPR)
    - Mid-clause pause duration (MCPD)
    - End-clause pause duration (ECPD)
    - Pause ratio (PR)
    - Pause duration (PD)
- Repair fluency
    - Disfluency ratio (DR)
- Composite
    - Speech rate (SR)
    - Mean length of run (MLR)

The output variable is logit value which is calculated by Many Facet Rasch model (i.e., raters' severity is removed).

Linear regressions are employed as prediction models.
The dataset is divided into training and testing at a ratio of 8:2 taking distribution into account.
For the model construction, because of relatively small data size, I utilize the stratified 5-fold cross validation technique.
The models are evaluated in terms of Pearon's correlation coefficient and $R^2$ score.

Before starting the evaluation, the following code block loads required packages and define global variables.

In [1]:
from typing import Tuple, Dict, Optional, List, Generator
from pathlib import Path

import numpy as np
import pandas as pd
import pickle as pkl
from dominance_analysis import Dominance

from utils.mfr import logit_2_rating

DATA_DIR = Path("/home/matsuura/Development/app/feature_extraction_api/experiment/data")

TASK = ["Arg_Oly", "Cartoon", "RtSwithoutRAA", "RtSwithRAA", "WoZ_Interview"]

TARGET_UF_MEASURES = [
    "articulation_rate", "mid_clause_pause_ratio", "end_clause_pause_ratio",
    "mid_clause_p-dur", "end_clause_p-dur", "pause_ratio", "mean_pause_duration",
    "dysfluency_ratio", "speech_rate", "mean_length_of_run"
]

RANDOM_STATE = 0

Loading BokehJS ...

---

## 2. Define Functions

This section defines functions for the analyses.
The following code block defines functions to load a dataset including UF measures and logit value.

In [2]:
def add_pause_ratio(df_uf: pd.DataFrame) -> pd.DataFrame:
    df_uf.loc[:, "pause_ratio"] = df_uf["mid_clause_pause_ratio"] + df_uf["end_clause_pause_ratio"]

    return df_uf

def calculate_mean_uf_measures(df_uf: pd.DataFrame) -> pd.DataFrame:
    data = []

    for uid in range(1, 86):
        uid = str(uid).zfill(3)

        mask = df_uf.index.str.startswith(uid)
        if mask.sum() == 0:
            continue

        df_uf_masked = df_uf[mask]
        mean_uf_measures = df_uf_masked.mean(axis=0, skipna=True)
        
        row = {
            "uid": uid   
        }
        for measure_name in df_uf.columns:
            row[measure_name] = mean_uf_measures[measure_name]
        data.append(row)

    df_uf_mean = pd.DataFrame.from_dict(data).set_index("uid", drop=True)

    return df_uf_mean

def load_dataset(task: str, manual: bool =True, task_facet: bool =False) -> Tuple[pd.DataFrame, pd.DataFrame]:
    if manual:
        uf_path = DATA_DIR / f"{task}/09_UF_Measures/uf_measures_manu_pruned.csv"
    else:
        uf_path = DATA_DIR / f"{task}/09_UF_Measures/uf_measures_auto_pruned.csv"

    df_uf = pd.read_csv(uf_path, index_col="uid")

    if task == "WoZ_Interview":
        df_uf = calculate_mean_uf_measures(df_uf)

    df_uf = add_pause_ratio(df_uf)

    X = df_uf[TARGET_UF_MEASURES].sort_index()

    pf_path = DATA_DIR / f"{task}/12_PF_Rating/pf_rating.csv"
    df_pf = pd.read_csv(pf_path)
    uid_list = df_pf["uid"].to_numpy()

    if task_facet:
        logit_path = pf_path.parent / "logit_all.csv"
        threshold_path = logit_path.parent / "threshold_all.csv"
    else:
        logit_path = pf_path.parent / "logit.csv"
        threshold_path = logit_path.parent / "threshold.csv"
    
    df_logit = pd.read_csv(logit_path, index_col=0)
    logit = df_logit["theta"].to_numpy()
    rating_list = logit_2_rating(df_logit["theta"], threshold_path)

    data = np.array([uid_list, logit, rating_list]).T
    columns = ["uid", "logit", "rating"]

    Y = pd.DataFrame(data, columns=columns).set_index("uid", drop=True).sort_index()
    
    return X, Y

The following code block define functions to conduct dominance analyses.

In [3]:
def fit_dominance_regression(
        X: pd.DataFrame,
        Y: pd.DataFrame,
        train_indices_list: List[np.ndarray]
) -> dict:
    Y = Y[["logit"]].astype(float)
    Y.index = X.index

    df_dataset = pd.merge(X, Y, left_index=True, right_index=True)
    result = {}

    best_trial = 0
    best_score = -100

    for trial, train_indices in enumerate(train_indices_list):
        df_train = df_dataset.iloc[train_indices, :]

        dominance_regression = Dominance(data=df_train, target="logit", objective=1)
        dominance_regression.incremental_rsquare()

        result[f"{trial} trial"] = dominance_regression

        complete_model_r2 = dominance_regression.model_rsquares[" ".join(TARGET_UF_MEASURES)]
        if complete_model_r2 > best_score:
            best_trial = trial

    result["best_trial"] = best_trial

    return result

In [4]:
def draw_relative_importance_graphs(result: dict, trial: Optional[int] =None) -> None:
    if trial is None:
        trial = result["best_trial"]

    trial_name = f"{trial} trial"

    dominance_regression = result[trial_name]
    dominance_regression.plot_incremental_rsquare()

def generate_dominance_statistics_table(
        result: dict,
        task: str,
        manual: bool,
        task_facet: bool
) -> pd.DataFrame:
    if manual:
        method = "manual"
    else:
        method = "auto"

    if task_facet:
        facet = "Rater, Task"
    else:
        facet = "Rater"

    dominance_stats_list = []
    for trial, dominance_regression in result.items():
        if isinstance(dominance_regression, int):
            continue

        df_dominance_stats = dominance_regression.dominance_stats()

        columns_origin = df_dominance_stats.columns.tolist()
        columns_new = ["name", "task", "method", "facet"] + columns_origin

        df_dominance_stats.loc[:, "name"] = trial
        df_dominance_stats.loc[:, "task"] = task
        df_dominance_stats.loc[:, "method"] = method
        df_dominance_stats.loc[:, "facet"] = facet

        df_dominance_stats = df_dominance_stats[columns_new]
        dominance_stats_list.append(df_dominance_stats)

    return pd.concat(dominance_stats_list)

---

## 3. Analyses

This section trains and evaluates linear regression models for each task.

In [5]:
with open("/home/matsuura/Development/app/feature_extraction_api/experiment/data/train_test_indices.pkl", "rb") as f:
    train_test_indices = pkl.load(f)

train_indices_list_mono = train_test_indices["monologue"]["train"]
train_indices_list_diag = train_test_indices["dialogue"]["train"]
df_eval_list = []

### 3.1. Arg Oly

**Manual Measures (Facet = Rater)**

In [6]:
task = "Arg_Oly"
manual = True
task_facet = False
X, Y = load_dataset(task, manual, task_facet)
result = fit_dominance_regression(X, Y, train_indices_list_mono)
df_dominance_stats = generate_dominance_statistics_table(result, task, manual, task_facet)
draw_relative_importance_graphs(result)
manual_best_trial = result["best_trial"]
df_eval_list.append(df_dominance_stats)
df_dominance_stats.head()

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.7653749993135849

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


100%|██████████| 10/10 [00:03<00:00,  3.32it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 34.65it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.7496046448803582

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


...

100%|██████████| 10/10 [00:03<00:00,  3.26it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 34.03it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################



,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
speech_rate,0 trial,Arg_Oly,manual,Rater,0.016283,0.654723,0.097923,0.145439,19.002291
mean_pause_duration,0 trial,Arg_Oly,manual,Rater,0.000038,0.5356,0.086236,0.122552,16.012053
mid_clause_p-dur,0 trial,Arg_Oly,manual,Rater,0.000742,0.451594,0.061365,0.094325,12.324061
pause_ratio,0 trial,Arg_Oly,manual,Rater,0.0,0.489561,0.042854,0.083239,10.875626
mid_clause_pause_ratio,0 trial,Arg_Oly,manual,Rater,-0.0,0.47325,0.038878,0.078427,10.246901


**Automatic Measures (Facet = Rater)**

In [7]:
task = "Arg_Oly"
manual = False
task_facet = False
X, Y = load_dataset(task, manual, task_facet)
result = fit_dominance_regression(X, Y, train_indices_list_mono)
df_dominance_stats = generate_dominance_statistics_table(result, task, manual, task_facet)
draw_relative_importance_graphs(result, manual_best_trial)
df_eval_list.append(df_dominance_stats)
df_dominance_stats.head()

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.7928286012035639

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


100%|██████████| 10/10 [00:03<00:00,  3.29it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 34.94it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.8020162479368934

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


...

100%|██████████| 10/10 [00:03<00:00,  3.25it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 38.87it/s]

#########################  Variable Dominance Calculation Done!!!!!  #########################



,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
speech_rate,0 trial,Arg_Oly,auto,Rater,0.004321,0.720845,0.111074,0.161376,20.354409
mean_pause_duration,0 trial,Arg_Oly,auto,Rater,0.011046,0.552266,0.083330,0.122995,15.513451
mid_clause_p-dur,0 trial,Arg_Oly,auto,Rater,0.006319,0.535512,0.073142,0.112697,14.214529
articulation_rate,0 trial,Arg_Oly,auto,Rater,0.002427,0.554723,0.068241,0.110308,13.913176
mid_clause_pause_ratio,0 trial,Arg_Oly,auto,Rater,0.0,0.427495,0.036143,0.071664,9.038985


**Manual Measures (Facet = Rater, Task)**

In [8]:
task = "Arg_Oly"
manual = True
task_facet = True
X, Y = load_dataset(task, manual, task_facet)
result = fit_dominance_regression(X, Y, train_indices_list_mono)
df_dominance_stats = generate_dominance_statistics_table(result, task, manual, task_facet)
draw_relative_importance_graphs(result)
manual_best_trial = result["best_trial"]
df_eval_list.append(df_dominance_stats)
df_dominance_stats.head()

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.8177856901725273

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


100%|██████████| 10/10 [00:03<00:00,  3.29it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 35.21it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.8375122311289428

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


...

100%|██████████| 10/10 [00:03<00:00,  3.28it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 38.30it/s]

#########################  Variable Dominance Calculation Done!!!!!  #########################



,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
mean_length_of_run,0 trial,Arg_Oly,manual,"Rater, Task",0.093963,0.661912,0.195699,0.232147,28.387211
speech_rate,0 trial,Arg_Oly,manual,"Rater, Task",0.00126,0.656024,0.121807,0.163174,19.95319
mean_pause_duration,0 trial,Arg_Oly,manual,"Rater, Task",0.008156,0.337048,0.059135,0.081828,10.006094
mid_clause_p-dur,0 trial,Arg_Oly,manual,"Rater, Task",0.00049,0.341494,0.058913,0.081329,9.945042
articulation_rate,0 trial,Arg_Oly,manual,"Rater, Task",0.003717,0.387793,0.040211,0.07132,8.721074


**Automatic Measures (Facet = Rater, Task)**

In [9]:
task = "Arg_Oly"
manual = True
task_facet = True
X, Y = load_dataset(task, manual, task_facet)
result = fit_dominance_regression(X, Y, train_indices_list_mono)
df_dominance_stats = generate_dominance_statistics_table(result, task, manual, task_facet)
draw_relative_importance_graphs(result, manual_best_trial)
df_eval_list.append(df_dominance_stats)
df_dominance_stats.head()

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.8177856901725273

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


100%|██████████| 10/10 [00:03<00:00,  3.28it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 35.44it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.8375122311289428

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


...

100%|██████████| 10/10 [00:03<00:00,  3.25it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 35.56it/s]

#########################  Variable Dominance Calculation Done!!!!!  #########################



,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
mean_length_of_run,0 trial,Arg_Oly,manual,"Rater, Task",0.093963,0.661912,0.195699,0.232147,28.387211
speech_rate,0 trial,Arg_Oly,manual,"Rater, Task",0.00126,0.656024,0.121807,0.163174,19.95319
mean_pause_duration,0 trial,Arg_Oly,manual,"Rater, Task",0.008156,0.337048,0.059135,0.081828,10.006094
mid_clause_p-dur,0 trial,Arg_Oly,manual,"Rater, Task",0.00049,0.341494,0.058913,0.081329,9.945042
articulation_rate,0 trial,Arg_Oly,manual,"Rater, Task",0.003717,0.387793,0.040211,0.07132,8.721074


### 3.2. Cartoon

**Manual Measures (Facet = Rater)**

In [10]:
task = "Cartoon"
manual = True
task_facet = False
X, Y = load_dataset(task, manual, task_facet)
result = fit_dominance_regression(X, Y, train_indices_list_mono)
df_dominance_stats = generate_dominance_statistics_table(result, task, manual, task_facet)
draw_relative_importance_graphs(result)
manual_best_trial = result["best_trial"]
df_eval_list.append(df_dominance_stats)
df_dominance_stats.head()

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.7896530090652514

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


100%|██████████| 10/10 [00:03<00:00,  3.27it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 35.20it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.7964826425930654

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


...

100%|██████████| 10/10 [00:03<00:00,  3.24it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 35.71it/s]

#########################  Variable Dominance Calculation Done!!!!!  #########################



,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
speech_rate,0 trial,Cartoon,manual,Rater,0.011272,0.728232,0.105049,0.15799,20.00751
mean_length_of_run,0 trial,Cartoon,manual,Rater,0.00172,0.601404,0.063280,0.110936,14.048742
mean_pause_duration,0 trial,Cartoon,manual,Rater,0.012163,0.5464,0.065616,0.108349,13.721149
pause_ratio,0 trial,Cartoon,manual,Rater,0.0,0.524048,0.032804,0.078648,9.959807
mid_clause_p-dur,0 trial,Cartoon,manual,Rater,0.009793,0.446548,0.041237,0.078624,9.956774


**Automatic Measures (Facet = Rater)**

In [11]:
task = "Cartoon"
manual = False
task_facet = False
X, Y = load_dataset(task, manual, task_facet)
result = fit_dominance_regression(X, Y, train_indices_list_mono)
df_dominance_stats = generate_dominance_statistics_table(result, task, manual, task_facet)
draw_relative_importance_graphs(result, manual_best_trial)
df_eval_list.append(df_dominance_stats)
df_dominance_stats.head()

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.7905995834749499

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


100%|██████████| 10/10 [00:03<00:00,  3.31it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 35.75it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.8036652633236795

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


...

100%|██████████| 10/10 [00:03<00:00,  3.26it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 39.13it/s]

#########################  Variable Dominance Calculation Done!!!!!  #########################



,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
speech_rate,0 trial,Cartoon,auto,Rater,0.025648,0.733814,0.158525,0.202766,25.647126
mean_pause_duration,0 trial,Cartoon,auto,Rater,0.000002,0.511785,0.072428,0.109121,13.802282
mean_length_of_run,0 trial,Cartoon,auto,Rater,0.001921,0.503732,0.072447,0.108523,13.726608
mid_clause_p-dur,0 trial,Cartoon,auto,Rater,0.000097,0.486787,0.067225,0.102468,12.960797
pause_ratio,0 trial,Cartoon,auto,Rater,0.0,0.37704,0.033250,0.064304,8.133564


**Manual Measures (Facet = Rater, Task)**

In [12]:
task = "Cartoon"
manual = True
task_facet = True
X, Y = load_dataset(task, manual, task_facet)
result = fit_dominance_regression(X, Y, train_indices_list_mono)
df_dominance_stats = generate_dominance_statistics_table(result, task, manual, task_facet)
draw_relative_importance_graphs(result)
manual_best_trial = result["best_trial"]
df_eval_list.append(df_dominance_stats)
df_dominance_stats.head()

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.7262374838957948

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


100%|██████████| 10/10 [00:03<00:00,  3.26it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 35.40it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.6731127932292569

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


...

100%|██████████| 10/10 [00:03<00:00,  3.30it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 39.24it/s]

#########################  Variable Dominance Calculation Done!!!!!  #########################



,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
mean_length_of_run,0 trial,Cartoon,manual,"Rater, Task",0.057167,0.537675,0.101161,0.140413,19.334263
speech_rate,0 trial,Cartoon,manual,"Rater, Task",0.003612,0.598817,0.086068,0.129098,17.77621
mean_pause_duration,0 trial,Cartoon,manual,"Rater, Task",0.034877,0.500185,0.076011,0.114315,15.740732
mid_clause_p-dur,0 trial,Cartoon,manual,"Rater, Task",0.016895,0.412077,0.047007,0.080503,11.084899
end_clause_p-dur,0 trial,Cartoon,manual,"Rater, Task",0.009784,0.32291,0.037371,0.063166,8.697751


**Automatic Measures (Facet = Rater, Task)**

In [13]:
task = "Cartoon"
manual = True
task_facet = True
X, Y = load_dataset(task, manual, task_facet)
result = fit_dominance_regression(X, Y, train_indices_list_mono)
df_dominance_stats = generate_dominance_statistics_table(result, task, manual, task_facet)
draw_relative_importance_graphs(result, manual_best_trial)
df_eval_list.append(df_dominance_stats)
df_dominance_stats.head()

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.7262374838957948

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


100%|██████████| 10/10 [00:03<00:00,  3.28it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 39.13it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.6731127932292569

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


...

100%|██████████| 10/10 [00:03<00:00,  3.33it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 35.86it/s]

#########################  Variable Dominance Calculation Done!!!!!  #########################



,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
mean_length_of_run,0 trial,Cartoon,manual,"Rater, Task",0.057167,0.537675,0.101161,0.140413,19.334263
speech_rate,0 trial,Cartoon,manual,"Rater, Task",0.003612,0.598817,0.086068,0.129098,17.77621
mean_pause_duration,0 trial,Cartoon,manual,"Rater, Task",0.034877,0.500185,0.076011,0.114315,15.740732
mid_clause_p-dur,0 trial,Cartoon,manual,"Rater, Task",0.016895,0.412077,0.047007,0.080503,11.084899
end_clause_p-dur,0 trial,Cartoon,manual,"Rater, Task",0.009784,0.32291,0.037371,0.063166,8.697751


### 3.3. RtSwithoutRAA

**Manual Measures (Facet = Rater)**

In [14]:
task = "RtSwithoutRAA"
manual = True
task_facet = False
X, Y = load_dataset(task, manual, task_facet)
result = fit_dominance_regression(X, Y, train_indices_list_mono)
df_dominance_stats = generate_dominance_statistics_table(result, task, manual, task_facet)
draw_relative_importance_graphs(result)
manual_best_trial = result["best_trial"]
df_eval_list.append(df_dominance_stats)
df_dominance_stats.head()

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.8164133383224826

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


100%|██████████| 10/10 [00:03<00:00,  3.22it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 36.45it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.831390899236239

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


...

100%|██████████| 10/10 [00:03<00:00,  3.27it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 39.08it/s]

#########################  Variable Dominance Calculation Done!!!!!  #########################



,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
speech_rate,0 trial,RtSwithoutRAA,manual,Rater,0.000017,0.729507,0.110925,0.161693,19.80524
mean_length_of_run,0 trial,RtSwithoutRAA,manual,Rater,0.013334,0.607427,0.095331,0.138341,16.945004
mean_pause_duration,0 trial,RtSwithoutRAA,manual,Rater,0.003904,0.611044,0.065912,0.114225,13.991042
mid_clause_p-dur,0 trial,RtSwithoutRAA,manual,Rater,0.000129,0.568141,0.051739,0.098218,12.030459
articulation_rate,0 trial,RtSwithoutRAA,manual,Rater,0.000099,0.506255,0.043964,0.085807,10.510186


**Automatic Measures (Facet = Rater)**

In [15]:
task = "RtSwithoutRAA"
manual = False
task_facet = False
X, Y = load_dataset(task, manual, task_facet)
result = fit_dominance_regression(X, Y, train_indices_list_mono)
df_dominance_stats = generate_dominance_statistics_table(result, task, manual, task_facet)
draw_relative_importance_graphs(result, manual_best_trial)
df_eval_list.append(df_dominance_stats)
df_dominance_stats.head()

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.8472349986570532

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


100%|██████████| 10/10 [00:03<00:00,  3.32it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 38.42it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.8707057053486262

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


...

100%|██████████| 10/10 [00:03<00:00,  3.28it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 38.47it/s]

#########################  Variable Dominance Calculation Done!!!!!  #########################



,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
speech_rate,0 trial,RtSwithoutRAA,auto,Rater,0.012379,0.781754,0.122182,0.177159,20.910201
mean_pause_duration,0 trial,RtSwithoutRAA,auto,Rater,0.007658,0.622216,0.093958,0.138153,16.306389
mid_clause_p-dur,0 trial,RtSwithoutRAA,auto,Rater,0.00249,0.516855,0.060683,0.10048,11.859812
mean_length_of_run,0 trial,RtSwithoutRAA,auto,Rater,0.002506,0.51997,0.057818,0.098502,11.62632
articulation_rate,0 trial,RtSwithoutRAA,auto,Rater,0.00052,0.459336,0.044022,0.081204,9.584531


**Manual Measures (Facet = Rater, Task)**

In [16]:
task = "RtSwithoutRAA"
manual = True
task_facet = True
X, Y = load_dataset(task, manual, task_facet)
result = fit_dominance_regression(X, Y, train_indices_list_mono)
df_dominance_stats = generate_dominance_statistics_table(result, task, manual, task_facet)
draw_relative_importance_graphs(result)
manual_best_trial = result["best_trial"]
df_eval_list.append(df_dominance_stats)
df_dominance_stats.head()

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.7459354062664845

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


100%|██████████| 10/10 [00:03<00:00,  3.31it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 38.50it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.7254452920292087

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


...

100%|██████████| 10/10 [00:03<00:00,  3.25it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 34.25it/s]

#########################  Variable Dominance Calculation Done!!!!!  #########################



,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
speech_rate,0 trial,RtSwithoutRAA,manual,"Rater, Task",0.000424,0.656138,0.147091,0.183329,24.57708
mean_length_of_run,0 trial,RtSwithoutRAA,manual,"Rater, Task",0.026346,0.601237,0.147155,0.180482,24.19538
mean_pause_duration,0 trial,RtSwithoutRAA,manual,"Rater, Task",0.017618,0.403703,0.055036,0.086161,11.5507
articulation_rate,0 trial,RtSwithoutRAA,manual,"Rater, Task",0.000382,0.398879,0.055198,0.084085,11.272408
mid_clause_p-dur,0 trial,RtSwithoutRAA,manual,"Rater, Task",0.00429,0.329597,0.031914,0.05892,7.898852


**Automatic Measures (Facet = Rater, Task)**

In [17]:
task = "RtSwithoutRAA"
manual = True
task_facet = True
X, Y = load_dataset(task, manual, task_facet)
result = fit_dominance_regression(X, Y, train_indices_list_mono)
df_dominance_stats = generate_dominance_statistics_table(result, task, manual, task_facet)
draw_relative_importance_graphs(result, manual_best_trial)
df_eval_list.append(df_dominance_stats)
df_dominance_stats.head()

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.7459354062664845

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


100%|██████████| 10/10 [00:03<00:00,  3.23it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 38.41it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.7254452920292087

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


...

100%|██████████| 10/10 [00:03<00:00,  3.28it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 39.56it/s]

#########################  Variable Dominance Calculation Done!!!!!  #########################



,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
speech_rate,0 trial,RtSwithoutRAA,manual,"Rater, Task",0.000424,0.656138,0.147091,0.183329,24.57708
mean_length_of_run,0 trial,RtSwithoutRAA,manual,"Rater, Task",0.026346,0.601237,0.147155,0.180482,24.19538
mean_pause_duration,0 trial,RtSwithoutRAA,manual,"Rater, Task",0.017618,0.403703,0.055036,0.086161,11.5507
articulation_rate,0 trial,RtSwithoutRAA,manual,"Rater, Task",0.000382,0.398879,0.055198,0.084085,11.272408
mid_clause_p-dur,0 trial,RtSwithoutRAA,manual,"Rater, Task",0.00429,0.329597,0.031914,0.05892,7.898852


### 3.4. RtSwithRAA

**Manual Measures (Facet = Rater)**

In [18]:
task = "RtSwithRAA"
manual = True
task_facet = False
X, Y = load_dataset(task, manual, task_facet)
result = fit_dominance_regression(X, Y, train_indices_list_mono)
df_dominance_stats = generate_dominance_statistics_table(result, task, manual, task_facet)
draw_relative_importance_graphs(result)
manual_best_trial = result["best_trial"]
df_eval_list.append(df_dominance_stats)
df_dominance_stats.head()

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.7638300250899479

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


100%|██████████| 10/10 [00:02<00:00,  3.34it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 34.90it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.7608945910752368

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


...

100%|██████████| 10/10 [00:03<00:00,  3.25it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 34.64it/s]

#########################  Variable Dominance Calculation Done!!!!!  #########################



,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
speech_rate,0 trial,RtSwithRAA,manual,Rater,0.026157,0.665112,0.121341,0.1662,21.758764
mean_pause_duration,0 trial,RtSwithRAA,manual,Rater,0.005156,0.499273,0.065276,0.102664,13.440682
mean_length_of_run,0 trial,RtSwithRAA,manual,Rater,0.000016,0.48987,0.063060,0.099437,13.018146
end_clause_p-dur,0 trial,RtSwithRAA,manual,Rater,0.000786,0.384867,0.057327,0.084427,11.053137
mid_clause_p-dur,0 trial,RtSwithRAA,manual,Rater,0.004929,0.432931,0.044212,0.079155,10.362964


**Automatic Measures (Facet = Rater)**

In [19]:
task = "RtSwithRAA"
manual = False
task_facet = False
X, Y = load_dataset(task, manual, task_facet)
result = fit_dominance_regression(X, Y, train_indices_list_mono)
df_dominance_stats = generate_dominance_statistics_table(result, task, manual, task_facet)
draw_relative_importance_graphs(result, manual_best_trial)
df_eval_list.append(df_dominance_stats)
df_dominance_stats.head()

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.7808868131141978

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


100%|██████████| 10/10 [00:02<00:00,  3.34it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 39.42it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.7602025624826835

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


...

100%|██████████| 10/10 [00:03<00:00,  3.33it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 38.49it/s]

#########################  Variable Dominance Calculation Done!!!!!  #########################



,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
speech_rate,0 trial,RtSwithRAA,auto,Rater,0.031026,0.699928,0.151764,0.194507,24.908447
mean_pause_duration,0 trial,RtSwithRAA,auto,Rater,0.000064,0.44312,0.073604,0.103202,13.215963
mid_clause_p-dur,0 trial,RtSwithRAA,auto,Rater,0.000773,0.412962,0.064355,0.092857,11.891274
mean_length_of_run,0 trial,RtSwithRAA,auto,Rater,0.000206,0.442103,0.057502,0.090232,11.555096
end_clause_p-dur,0 trial,RtSwithRAA,auto,Rater,0.009552,0.230352,0.059324,0.07145,9.149836


**Manual Measures (Facet = Rater, Task)**

In [20]:
task = "RtSwithRAA"
manual = True
task_facet = True
X, Y = load_dataset(task, manual, task_facet)
result = fit_dominance_regression(X, Y, train_indices_list_mono)
df_dominance_stats = generate_dominance_statistics_table(result, task, manual, task_facet)
draw_relative_importance_graphs(result)
manual_best_trial = result["best_trial"]
df_eval_list.append(df_dominance_stats)
df_dominance_stats.head()

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.7409424534479809

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


100%|██████████| 10/10 [00:03<00:00,  3.32it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 39.02it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.688629762537436

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


...

100%|██████████| 10/10 [00:03<00:00,  3.31it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 36.87it/s]

#########################  Variable Dominance Calculation Done!!!!!  #########################



,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
mean_length_of_run,0 trial,RtSwithRAA,manual,"Rater, Task",0.044232,0.560127,0.138047,0.170873,23.0616
speech_rate,0 trial,RtSwithRAA,manual,"Rater, Task",0.000422,0.622838,0.122319,0.160181,21.618571
mean_pause_duration,0 trial,RtSwithRAA,manual,"Rater, Task",0.006955,0.398213,0.049898,0.080435,10.855763
end_clause_p-dur,0 trial,RtSwithRAA,manual,"Rater, Task",0.000616,0.310507,0.044497,0.06671,9.003392
mid_clause_p-dur,0 trial,RtSwithRAA,manual,"Rater, Task",0.003837,0.353663,0.034699,0.063509,8.571378


**Automatic Measures (Facet = Rater, Task)**

In [21]:
task = "RtSwithRAA"
manual = True
task_facet = True
X, Y = load_dataset(task, manual, task_facet)
result = fit_dominance_regression(X, Y, train_indices_list_mono)
df_dominance_stats = generate_dominance_statistics_table(result, task, manual, task_facet)
draw_relative_importance_graphs(result, manual_best_trial)
df_eval_list.append(df_dominance_stats)
df_dominance_stats.head()

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.7409424534479809

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


100%|██████████| 10/10 [00:03<00:00,  3.30it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 34.58it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.688629762537436

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


...

100%|██████████| 10/10 [00:03<00:00,  3.30it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 38.16it/s]

#########################  Variable Dominance Calculation Done!!!!!  #########################



,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
mean_length_of_run,0 trial,RtSwithRAA,manual,"Rater, Task",0.044232,0.560127,0.138047,0.170873,23.0616
speech_rate,0 trial,RtSwithRAA,manual,"Rater, Task",0.000422,0.622838,0.122319,0.160181,21.618571
mean_pause_duration,0 trial,RtSwithRAA,manual,"Rater, Task",0.006955,0.398213,0.049898,0.080435,10.855763
end_clause_p-dur,0 trial,RtSwithRAA,manual,"Rater, Task",0.000616,0.310507,0.044497,0.06671,9.003392
mid_clause_p-dur,0 trial,RtSwithRAA,manual,"Rater, Task",0.003837,0.353663,0.034699,0.063509,8.571378


### 3.5. WoZ Interview

**Manual Measures (Facet = Rater)**

In [22]:
task = "WoZ_Interview"
manual = True
task_facet = False
X, Y = load_dataset(task, manual, task_facet)
result = fit_dominance_regression(X, Y, train_indices_list_diag)
df_dominance_stats = generate_dominance_statistics_table(result, task, manual, task_facet)
draw_relative_importance_graphs(result)
manual_best_trial = result["best_trial"]
df_eval_list.append(df_dominance_stats)
df_dominance_stats.head()

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.6661102355321984

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


100%|██████████| 10/10 [00:03<00:00,  3.29it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 35.78it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.6324875973366455

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


...

100%|██████████| 10/10 [00:03<00:00,  3.33it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 34.78it/s]

#########################  Variable Dominance Calculation Done!!!!!  #########################



,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
mean_length_of_run,0 trial,WoZ_Interview,manual,Rater,0.060649,0.476174,0.128790,0.156714,23.526811
speech_rate,0 trial,WoZ_Interview,manual,Rater,0.014621,0.498066,0.115966,0.144042,21.624283
articulation_rate,0 trial,WoZ_Interview,manual,Rater,0.002017,0.518289,0.105513,0.136441,20.483298
mid_clause_pause_ratio,0 trial,WoZ_Interview,manual,Rater,0.0,0.325448,0.036535,0.061773,9.273695
pause_ratio,0 trial,WoZ_Interview,manual,Rater,0.0,0.291475,0.032097,0.054825,8.230638


**Automatic Measures (Facet = Rater)**

In [23]:
task = "WoZ_Interview"
manual = False
task_facet = False
X, Y = load_dataset(task, manual, task_facet)
result = fit_dominance_regression(X, Y, train_indices_list_diag)
df_dominance_stats = generate_dominance_statistics_table(result, task, manual, task_facet)
draw_relative_importance_graphs(result, manual_best_trial)
df_eval_list.append(df_dominance_stats)
df_dominance_stats.head()

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.6766729014557926

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


100%|██████████| 10/10 [00:03<00:00,  3.29it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 34.88it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

********************  R-Squared of Complete Model :  ********************
R Squared : 0.59485913479353

Selecting 10 Best Predictors for the Model
Selected Predictors :  ['articulation_rate', 'mid_clause_pause_ratio', 'end_clause_pause_ratio', 'mid_clause_p-dur', 'end_clause_p-dur', 'pause_ratio', 'mean_pause_duration', 'dysfluency_ratio', 'speech_rate', 'mean_length_of_run']

Creating models for 1023 possible combinations of 10 features :


...

100%|██████████| 10/10 [00:03<00:00,  3.23it/s]


#########################  Model Training Done!!!!!  #########################

#########################  Calculating Variable Dominances  #########################


100%|██████████| 9/9 [00:00<00:00, 35.56it/s]


#########################  Variable Dominance Calculation Done!!!!!  #########################



,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
mean_length_of_run,0 trial,WoZ_Interview,auto,Rater,0.08228,0.478562,0.180724,0.200663,29.65443
mid_clause_pause_ratio,0 trial,WoZ_Interview,auto,Rater,-0.0,0.368504,0.080046,0.100887,14.909256
pause_ratio,0 trial,WoZ_Interview,auto,Rater,0.0,0.349441,0.074465,0.094516,13.967758
mean_pause_duration,0 trial,WoZ_Interview,auto,Rater,0.109354,0.102085,0.063346,0.071821,10.613834
mid_clause_p-dur,0 trial,WoZ_Interview,auto,Rater,0.102614,0.078998,0.054508,0.061768,9.128148


### 3.6. Summary

In [24]:
df_all_eval = pd.concat(df_eval_list)
df_all_eval.to_csv("/home/matsuura/Development/app/feature_extraction_api/experiment/data/dominance_analysis_all_results.csv")
df_all_eval

,name,task,method,facet,Interactional Dominance,Individual Dominance,Average Partial Dominance,Total Dominance,Percentage Relative Importance
speech_rate,0 trial,Arg_Oly,manual,Rater,0.016283,0.654723,0.097923,0.145439,19.002291
mean_pause_duration,0 trial,Arg_Oly,manual,Rater,0.000038,0.5356,0.086236,0.122552,16.012053
mid_clause_p-dur,0 trial,Arg_Oly,manual,Rater,0.000742,0.451594,0.061365,0.094325,12.324061
pause_ratio,0 trial,Arg_Oly,manual,Rater,0.0,0.489561,0.042854,0.083239,10.875626
mid_clause_pause_ratio,0 trial,Arg_Oly,manual,Rater,-0.0,0.47325,0.038878,0.078427,10.246901
...,...,...,...,...,...,...,...,...,...
mid_clause_p-dur,4 trial,WoZ_Interview,auto,Rater,0.044041,0.139327,0.043450,0.053096,8.367592
articulation_rate,4 trial,WoZ_Interview,auto,Rater,0.005946,0.079549,0.019490,0.024141,3.804483
dysfluency_ratio,4 trial,WoZ_Interview,auto,Rater,0.000015,0.120935,0.013505,0.022899,3.608714
speech_rate,4 trial,WoZ_Interview,auto,Rater,0.0028,0.090926,0.015262,0.021582,3.401208
